# Bike Rentals Analysis
***
This analysis was conducted as a project from Codecademy’s Data Engineer Career
Path. The course provided the source data and the following prompt to start the project:

> A bike rental company has asked you to create a database to help their analysts understand the 
effects of weather on bike rentals. You’ve been given a year of bike rental data from the company 
and you’ll source weather data from the government. You’ll need to clean and validate both data 
sets, design a relational PostgreSQL database to store the data, and develop views for the 
database to assist the analytics team.

## (1.) Prepare Data and Environment (VSCode)
<img src="images/project_start_folder.png" width="600">
<br>
<br>
Setup Jupyter Notebook in VSCode with a virtual environment (.venv) for the project <br>
<img src="images/vscode_folder.png" width="300">
<br>
<br>

## (2.) Import, Inspect, Clean, and Export files (Python)

### Import Files

In [4]:
import pandas as pd
import numpy as np
import glob

# use glob to collect csv's and concat them to a single csv
files = glob.glob("project_starter_kit/data/JC-*-citibike-tripdata.csv")
df_list = []
for filename in files:
    data = pd.read_csv(filename)
    df_list.append(data)

bikedata = pd.concat(df_list)

weather = pd.read_csv("project_starter_kit/data/newark_airport_2016.csv")
bikedata.head()

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender
0,361,2016-02-01 00:31:18,2016-02-01 00:37:19,3202,Newport PATH,40.727224,-74.033759,3203,Hamilton Park,40.727596,-74.044247,24393,Subscriber,1975.0,1
1,297,2016-02-01 01:55:05,2016-02-01 02:00:02,3195,Sip Ave,40.730743,-74.063784,3194,McGinley Square,40.725340,-74.067622,24394,Subscriber,1985.0,2
2,1155,2016-02-01 02:40:05,2016-02-01 02:59:20,3183,Exchange Place,40.716247,-74.033459,3210,Pershing Field,40.742677,-74.051789,24676,Subscriber,1976.0,1
3,1769,2016-02-01 05:11:28,2016-02-01 05:40:58,3214,Essex Light Rail,40.712774,-74.036486,3203,Hamilton Park,40.727596,-74.044247,24700,Subscriber,1974.0,2
4,935,2016-02-01 05:48:24,2016-02-01 06:03:59,3203,Hamilton Park,40.727596,-74.044247,3214,Essex Light Rail,40.712774,-74.036486,24639,Subscriber,1974.0,2


### Inspect the Data

In [517]:
pd.set_option('display.max_rows', 1000) 
pd.set_option('display.max_columns',504)
pd.set_option('display.width',1000)

# print(bikedata.head(10))
# print(weather.head(10))


#bikedata -----------------------------------
print("len(bikedata): " + str(len(bikedata)))
bike_dup = bikedata.duplicated()
#print(bike_dup.value_counts())

print("number of bikes: " + str(len(bikedata['Bike ID'].unique())))

print("min(start_time): " + str(bikedata['Start Time'].min()))
print("max(start_time): " + str(bikedata['Start Time'].max()))
print("youngest rider: " + str(2016 - bikedata['Birth Year'].max()) + " years old")
print("oldest rider: " + str(2016 - bikedata['Birth Year'].min()) + " years old")
print("avg trip duration: " + str(round(bikedata['Trip Duration'].mean() / 60, 2)) + " minutes")


# The oldest person known to have lived in New Jersey in 2016 was Adele Dunlap, 
# who at the time was the oldest living person in the United States. She was born on December 12, 1902, 
# and lived in Hunterdon County, New Jersey. In 2016, she turned 114 years old. She passed away on 
# February 5, 2017, at the age of 114 years and 51 days.


#too_old = bikedata[bikedata.birth_year < 1902]


# station_names = bikedata['Start Station Name'].unique()
# station_names

# 9’s in a field (e.g.9999) indicate missing data or data that has not been received.

bikedata.head()


len(bikedata): 247584
number of bikes: 566
min(start_time): 2016-01-01 00:02:52
max(start_time): 2016-12-31 23:44:50
youngest rider: 16.0 years old
oldest rider: 116.0 years old
avg trip duration: 14.76 minutes


,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender
0,361,2016-02-01 00:31:18,2016-02-01 00:37:19,3202,Newport PATH,40.727224,-74.033759,3203,Hamilton Park,40.727596,-74.044247,24393,Subscriber,1975.0,1
1,297,2016-02-01 01:55:05,2016-02-01 02:00:02,3195,Sip Ave,40.730743,-74.063784,3194,McGinley Square,40.725340,-74.067622,24394,Subscriber,1985.0,2
2,1155,2016-02-01 02:40:05,2016-02-01 02:59:20,3183,Exchange Place,40.716247,-74.033459,3210,Pershing Field,40.742677,-74.051789,24676,Subscriber,1976.0,1
3,1769,2016-02-01 05:11:28,2016-02-01 05:40:58,3214,Essex Light Rail,40.712774,-74.036486,3203,Hamilton Park,40.727596,-74.044247,24700,Subscriber,1974.0,2
4,935,2016-02-01 05:48:24,2016-02-01 06:03:59,3203,Hamilton Park,40.727596,-74.044247,3214,Essex Light Rail,40.712774,-74.036486,24639,Subscriber,1974.0,2


### Clean the Data

In [518]:
#adjust column names
bikedata.columns = bikedata.columns.str.lower().str.replace(' ','_')
weather.columns = weather.columns.str.lower()

#get rid of riders that are too old
bikedata = bikedata[bikedata.birth_year >= 1902]

#edit gender column: use strings 'M', 'F', and 'U' instead of int 1, 2, and 0
def gender(num):
    if num == 0:
        return 'U'
    elif num == 1:
        return 'M'
    else:
        return 'F'

bikedata.gender = bikedata.gender.astype(int).apply(gender)

# change birth year to int 
bikedata.birth_year = bikedata.birth_year.astype(int)

#create stations df 
bikedata = bikedata.rename(columns = {'start_station_latitude': 'start_latitude', 
                                      'start_station_longitude': 'start_longitude', 
                                      'end_station_latitude': 'end_latitude', 
                                      'end_station_longitude': 'end_longitude'})

start_stations = bikedata[['start_station_id', 'start_station_name', 'start_latitude','start_longitude']].drop_duplicates() # create start_stations df

start_stations.columns = start_stations.columns.str.replace('start_', '') # modify column names

end_stations = bikedata[['end_station_id', 'end_station_name', 'end_latitude','end_longitude']].drop_duplicates() # create end_stations df
end_stations.columns = end_stations.columns.str.replace('end_', '')

stations = pd.concat([start_stations, end_stations]).drop_duplicates().reset_index(drop=True) # use start_stations and end_stations to create stations df, making sure all stations in the data are included.
stations.columns = stations.columns.str.replace('station_', '') # modify column names

# create trip_data df 
trip_data = bikedata[['bike_id', 'start_station_id', 'start_time', 'end_station_id', 'stop_time', 'trip_duration', 'user_type', 'birth_year', 'gender']]

# clean weather table 
weather = weather.drop(columns = ['pgtm', 'tsun'])
weather.columns.str.lower()
weather = weather.rename(columns = {'awnd': 'awnd_mph', 'snwd': 'snw_dpth'})
weather = weather.drop(columns = ['station', 'name'])

In [520]:
trip_data.head(20)

,bike_id,start_station_id,start_time,end_station_id,stop_time,trip_duration,user_type,birth_year,gender
0,24393,3202,2016-02-01 00:31:18,3203,2016-02-01 00:37:19,361,Subscriber,1975,M
1,24394,3195,2016-02-01 01:55:05,3194,2016-02-01 02:00:02,297,Subscriber,1985,F
2,24676,3183,2016-02-01 02:40:05,3210,2016-02-01 02:59:20,1155,Subscriber,1976,M
3,24700,3214,2016-02-01 05:11:28,3203,2016-02-01 05:40:58,1769,Subscriber,1974,F
4,24639,3203,2016-02-01 05:48:24,3214,2016-02-01 06:03:59,935,Subscriber,1974,F
5,24498,3212,2016-02-01 05:52:18,3185,2016-02-01 06:01:47,569,Subscriber,1984,F
6,24621,3214,2016-02-01 06:01:32,3186,2016-02-01 06:06:25,293,Subscriber,1974,M
7,24560,3209,2016-02-01 06:01:51,3186,2016-02-01 06:06:03,252,Subscriber,1991,M
8,24386,3209,2016-02-01 06:20:48,3186,2016-02-01 06:25:04,256,Subscriber,1989,F
9,24702,3184,2016-02-01 06:36:42,3183,2016-02-01 06:38:17,94,Subscriber,1990,F


In [521]:
weather.head(20)

,date,awnd_mph,prcp,snow,snw_dpth,tavg,tmax,tmin,wdf2,wdf5,wsf2,wsf5
0,2016-01-01,12.75,0.00,0.0,0.0,41,43,34,270,280.0,25.9,35.1
1,2016-01-02,9.40,0.00,0.0,0.0,36,42,30,260,260.0,21.0,25.1
2,2016-01-03,10.29,0.00,0.0,0.0,37,47,28,270,250.0,23.9,30.0
3,2016-01-04,17.22,0.00,0.0,0.0,32,35,14,330,330.0,25.9,33.1
4,2016-01-05,9.84,0.00,0.0,0.0,19,31,10,360,350.0,25.1,31.1
5,2016-01-06,5.37,0.00,0.0,0.0,28,42,15,230,250.0,12.1,16.1
6,2016-01-07,3.36,0.00,0.0,0.0,35,46,24,20,360.0,8.9,10.1
7,2016-01-08,8.05,0.00,0.0,0.0,38,45,31,20,30.0,14.1,16.1
8,2016-01-09,6.71,0.01,0.0,0.0,44,48,38,60,70.0,13.0,17.0
9,2016-01-10,15.43,1.77,0.0,0.0,53,65,39,260,270.0,36.0,42.9


### Write to CSV

In [522]:
trip_data.to_csv('trip_data.csv', index=False)
weather.to_csv('weather.csv', index=False)
stations.to_csv('stations.csv', index=False)

## (3.) Create a Database Schema (Lucid Chart)
<img src='images/schema_screenshot.png' width="600">
<br>

## (4.) Import Data and Create Views for Analysis (PostgreSQL)

### Import Data
First I imported the files I had cleaned with Python from my project folder:

In [ ]:
-- import stations.csv into the stations table
COPY stations(
  id,
  name,
  latitude,
  longitude
)
FROM '/python_cleanup_export/stations.csv'
DELIMITER ','
CSV HEADER;

-- import weather.csv into the weather table
COPY weather(
  date,
  awnd_mph,
  prcp,
  snow,
  snw_dpth,
  tavg,
  tmax,
  tmin,
  wdf2,
  wdf5,
  wsf2,
  wsf5
)
FROM '/python_cleanup_export/weather.csv'
DELIMITER ','
CSV HEADER;

-- import trip_data.csv into the trip_data table
COPY trip_data(
  trip_id,
  bike_id,
  start_station_id,
  start_time,
  end_station_id,
  stop_time,
  trip_duration,
  user_type,
  birth_year,
  gender
)
FROM '/python_cleanup_export/trip_data.csv'
DELIMITER ','
CSV HEADER;

### Create Views

<details>
  <summary>av_rides_hourly</summary>
  <pre><code class="language-sql">
CREATE VIEW avg_rides_hourly AS (

  WITH 
  hours AS (
    SELECT DISTINCT 
      TO_CHAR(start_time, 'HH24') AS hour
    FROM trip_data
    ORDER BY 1
  ),

  assign_trips AS (
    SELECT 
      TO_CHAR(t.start_time, 'MM/DD/YYYY') AS date,
      h.hour,
      t.trip_id
    FROM trip_data t
    JOIN hours h ON h.hour = TO_CHAR(t.start_time, 'HH24')
    ORDER BY 1
  ),
  
  aggr_trips AS (
    SELECT 
      date,
      hour,
      COUNT(trip_id) AS count
    FROM assign_trips
    GROUP BY date, hour
    ORDER BY date, hour
  )
  
  SELECT DISTINCT
    hour, 
    ROUND(AVG(count), 0) AS avg_rides
  FROM aggr_trips
  GROUP BY hour
  ORDER BY hour
);
  </code></pre>
</details>

<details>
  <summary>daily_trips_weather</summary>
  <pre><code class="language-sql">
CREATE VIEW daily_trips_weather AS (
  SELECT DISTINCT
    w.date,
    TO_CHAR(w.date, 'Day') AS weekday,
    COUNT(t.trip_id) OVER (
      PARTITION BY date
      ) AS trips,
  	ROUND(AVG(t.trip_duration) OVER (
      PARTITION BY date
    	),2) AS avg_trip_duration,
    w.awnd_mph,
    w.prcp,
    w.snow,
    w.snw_dpth,
    w.tavg,
    w.tmin,
    w.tmax,
    w.wdf2,
    w.wdf5,
    w.wsf2,
    w.wsf5
  FROM trip_data t
  JOIN weather w ON DATE(w.date) = DATE(t.start_time)
  ORDER BY 1
);
  </code></pre>
</details>

<details>
  <summary>travel_directions</summary>
  <pre><code class="language-sql">
CREATE VIEW travel_directions AS (

WITH coords AS (
    SELECT 
        t.trip_id,
        d.latitude AS lat1, 
        d.longitude AS lon1,
        a.latitude AS lat2,
        a.longitude AS lon2
    FROM trip_data t
    JOIN stations d ON t.start_station_id = d.id
    JOIN stations a ON t.end_station_id = a.id
),

directions AS (
  SELECT 
      trip_id,
      DEGREES(
          ATAN2(
              SIN(RADIANS(lon2 - lon1)) * COS(RADIANS(lat2)),
              COS(RADIANS(lat1)) * SIN(RADIANS(lat2)) - 
              SIN(RADIANS(lat1)) * COS(RADIANS(lat2)) * COS(RADIANS(lon2 - lon1))
          )
      ) AS bearing_degrees
  FROM coords
)

SELECT
	trip_id,
  bearing_degrees,
	CASE
    WHEN bearing_degrees >= 337.5 OR bearing_degrees < 22.5 THEN 'N'
    WHEN bearing_degrees >= 22.5 AND bearing_degrees < 67.5 THEN 'NE'
    WHEN bearing_degrees >= 67.5 AND bearing_degrees < 112.5 THEN 'E'
    WHEN bearing_degrees >= 112.5 AND bearing_degrees < 157.5 THEN 'SE'
    WHEN bearing_degrees >= 157.5 AND bearing_degrees < 202.5 THEN 'S'
    WHEN bearing_degrees >= 202.5 AND bearing_degrees < 247.5 THEN 'SW'
    WHEN bearing_degrees >= 247.5 AND bearing_degrees < 292.5 THEN 'W'
    WHEN bearing_degrees >= 292.5 AND bearing_degrees < 337.5 THEN 'NW'
  END AS direction
FROM directions
  
);

  </code></pre>
</details>

<details>
  <summary>travel_vs_wind</summary>
  <pre><code class="language-sql">
CREATE VIEW travel_vs_wind AS (
  
WITH cte AS (
  SELECT 
    w.date,
    d.trip_id,
    concat(s.name, ' to ', e.name) AS trip,
    t.trip_duration,
    d.direction as trip_direction,
    w.wdf2,
    w.wsf2,
    CASE
      WHEN w.wdf2 >= 337.5 OR w.wdf2 < 22.5 THEN 'N'
      WHEN w.wdf2 >= 22.5 AND w.wdf2 < 67.5 THEN 'NE'
      WHEN w.wdf2 >= 67.5 AND w.wdf2 < 112.5 THEN 'E'
      WHEN w.wdf2 >= 112.5 AND w.wdf2 < 157.5 THEN 'SE'
      WHEN w.wdf2 >= 157.5 AND w.wdf2 < 202.5 THEN 'S'
      WHEN w.wdf2 >= 202.5 AND w.wdf2 < 247.5 THEN 'SW'
      WHEN w.wdf2 >= 247.5 AND w.wdf2 < 292.5 THEN 'W'
      WHEN w.wdf2 >= 292.5 AND w.wdf2 < 337.5 THEN 'NW'
    END AS wdf2_direction,
    w.wdf5,
    w.wsf5,
    CASE
      WHEN w.wdf5 >= 337.5 OR w.wdf5 < 22.5 THEN 'N'
      WHEN w.wdf5 >= 22.5 AND w.wdf5 < 67.5 THEN 'NE'
      WHEN w.wdf5 >= 67.5 AND w.wdf5 < 112.5 THEN 'E'
      WHEN w.wdf5 >= 112.5 AND w.wdf5 < 157.5 THEN 'SE'
      WHEN w.wdf5 >= 157.5 AND w.wdf5 < 202.5 THEN 'S'
      WHEN w.wdf5 >= 202.5 AND w.wdf5 < 247.5 THEN 'SW'
      WHEN w.wdf5 >= 247.5 AND w.wdf5 < 292.5 THEN 'W'
      WHEN w.wdf5 >= 292.5 AND w.wdf5 < 337.5 THEN 'NW'
    END AS wdf5_direction
  FROM travel_directions d
  JOIN trip_data t ON t.trip_id = d.trip_id
  JOIN weather w ON DATE(w.date) = DATE(t.start_time)
  JOIN stations s ON t.start_station_id = s.id
  JOIN stations e ON t.end_station_id = e.id
  )

SELECT
  date,
  trip_id,
  trip,
  trip_duration,
  trip_direction,
  wdf2,
  wsf2,
  wdf2_direction,
  wdf5,
  wsf5,
  wdf5_direction,
  CASE
  	WHEN
  		(trip_direction = 'N' AND (wdf2_direction = 'S' OR wdf5_direction = 'S'))
  OR 	(trip_direction = 'NW' AND (wdf2_direction = 'SE' OR wdf5_direction = 'SE'))
  OR 	(trip_direction = 'W' AND (wdf2_direction = 'E' OR wdf5_direction = 'E'))
  OR 	(trip_direction = 'SW' AND (wdf2_direction = 'NE' OR wdf5_direction = 'NE'))
  OR 	(trip_direction = 'S' AND (wdf2_direction = 'N' OR wdf5_direction = 'N'))
  OR 	(trip_direction = 'SE' AND (wdf2_direction = 'NW' OR wdf5_direction = 'NW'))
  OR 	(trip_direction = 'E' AND (wdf2_direction = 'W' OR wdf5_direction = 'W'))
  OR 	(trip_direction = 'NE' AND (wdf2_direction = 'SW' OR wdf5_direction = 'SW'))
  THEN 1
  ELSE 0
  END AS against_wind
FROM cte
WHERE trip_duration <= 28800
ORDER BY date
);
  </code></pre>
</details>

<details>
  <summary>weather_and_trips_duration</summary>
  <pre><code class="language-sql">
CREATE VIEW weather_trips_and_duration AS (
WITH cte AS (
  SELECT DISTINCT
    w.date,
    t.trip_id,
    t.trip_duration,
    CASE
      WHEN w.prcp > 0
      THEN 1
      ELSE 0
    END AS is_rain,
    CASE
      WHEN (w.snow > 0 OR w.snw_dpth > 2)
      THEN 1
      ELSE 0
    END AS is_snow,
    CASE
      WHEN (w.wsf2 > 20 OR w.wsf5 > 20)
      THEN 1
      ELSE 0
    END AS is_windy
  FROM trip_data t
  JOIN weather w ON DATE(w.date) = DATE(t.start_time)
  ORDER BY 1
  ),

clear AS (
  SELECT
  	date,
  	COUNT(trip_id) AS ct_trips,
  	ROUND(AVG(trip_duration),2) AS avg_duration
 	FROM cte
  WHERE (is_rain = 0 AND is_snow = 0 AND is_windy = 0)
  GROUP BY date
  ORDER BY date
	),
  
rain AS (
  SELECT
  	date,
  	COUNT(trip_id) AS ct_trips,
  	ROUND(AVG(trip_duration),2) AS avg_duration
 	FROM cte
  WHERE is_rain = 1
  GROUP BY date
  ORDER BY date
	),

snow AS (
  SELECT
  	date,
  	COUNT(trip_id) AS ct_trips,
  	ROUND(AVG(trip_duration),2) AS avg_duration
 	FROM cte
  WHERE is_snow = 1
  GROUP BY date
  ORDER BY date
	),

windy AS (
  SELECT
  	date,
  	COUNT(trip_id) AS ct_trips,
  	ROUND(AVG(trip_duration),2) AS avg_duration
 	FROM cte
  WHERE is_windy = 1
  GROUP BY date
  ORDER BY date
	)

SELECT
	ROUND(AVG(clear.ct_trips),2) AS avg_clear_trips,
  ROUND(AVG(clear.avg_duration),2) AS avg_clear_duration,
  ROUND(AVG(rain.ct_trips),2) AS avg_rain_trips,
  ROUND(AVG(rain.avg_duration),2) AS avg_rain_duration,
  ROUND(AVG(snow.ct_trips),2) AS avg_snow_trips,
  ROUND(AVG(snow.avg_duration),2) AS avg_snow_duration,
  ROUND(AVG(windy.ct_trips),2) AS avg_windy_trips,
  ROUND(AVG(windy.avg_duration),2) AS avg_windy_duration
FROM clear, rain, snow, windy

);
  </code></pre>
</details>

### Export Views as CSV

In [ ]:
COPY (SELECT * FROM avg_rides_hourly) TO '/Users/nate/Data_Projects/Bike Rental Portfolio Project/avg_rides_hourly.csv' DELIMITER ',' CSV HEADER;
COPY (SELECT * FROM daily_trips_weather) TO '/Users/nate/Data_Projects/Bike Rental Portfolio Project/daily_trips_weather.csv' DELIMITER ',' CSV HEADER;
COPY (SELECT * FROM travel_directions) TO '/Users/nate/Data_Projects/Bike Rental Portfolio Project/travel_directions.csv' DELIMITER ',' CSV HEADER;
COPY (SELECT * FROM travel_vs_wind) TO '/Users/nate/Data_Projects/Bike Rental Portfolio Project/travel_vs_wind.csv' DELIMITER ',' CSV HEADER;
COPY (SELECT * FROM weather_trips_and_duration) TO '/Users/nate/Data_Projects/Bike Rental Portfolio Project/weather_trips_and_duration.csv' DELIMITER ',' CSV HEADER;

## Create Data Visualizations (Tableau)
I wanted to create some quick visualizations just to put the data to use so I put together two simple visuals with Tableau:

Using a top 10 filter and counting the records we can see what trips were most popular in 2016.
<br>
<img src='images/top_10_trips_viz.png' width="400">
<br>

My next idea was to use a visual to see if there was a significant affect on ride duration if riders were traveling against strong oncoming wind. Unfortunately, it didn’t seem like there was any affect. In fact, the average trip duration was longer when there was no oncoming wind.
<br>
<img src='images/riding_against_wind_viz.png' width="400">
<br>

Lastly, I used the weather_trips_and_duration view to visualize the effect that the weather has on the number of rides daily and their average duration.
From the graph you can see that customers are far less likely to ride in the rain or snow than they are on clear or windy days. Unlike the graph above, it does seem here that wind has a large effect on ride duration, regardless of wind being head-on or not.
<br>
<img src='images/weather_vs_trips_viz.png' width="400">
<br>

This CitiBike project analyzed a year's worth of bike rental data to understand the impact of weather on riding patterns. Using Python for data processing and PostgreSQL for database management, I conducted a comprehensive analysis of the dataset. Key findings include identifying the most popular bike trips in 2016 and, the effect of headwinds on trip duration, and the overall effect on weather on daily ride activity. This was a great exercise to practice my skills in preparing data for analysis, and I’m looking forward to what else Codecademy has to offer in its Data Engineering Career Path.